In [ ]:
import requests
from requests import Response
url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'
response = requests.get(url)
print(response.text)

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# 尋找帶有特定 title 屬性的表格
rate_table = soup.find('table', {'title': '牌告匯率'})

# 印出表格內容
#print(rate_table)

# 取得所有幣別和現金匯率資料
currency_rows = rate_table.tbody.find_all('tr')  # 跳過表頭行
#print(currency_rows)

for row in currency_rows:
    #print(row)
    print(row.find("td", attrs={"data-table": "幣別"}).find("div", class_="visible-phone").text.strip())
    # 取得幣別名稱 (移除多餘空白)
    # currency = row.find('div', class_='visible-phone').text.strip()
    
    # # 取得現金買入匯率
    # cash_buy = row.find_all('td')[1].text.strip()
    # # 取得現金賣出匯率
    # cash_sell = row.find_all('td')[2].text.strip()
    
    # # 轉換匯率值
    # cash_buy = round(float(cash_buy), 2) if cash_buy else None
    # cash_sell = round(float(cash_sell), 2) if cash_sell else None
    
    # print(f"幣別: {currency}")
    # print(f"現金買入: {cash_buy}")
    # print(f"現金賣出: {cash_sell}")
    # print("-" * 30)

美金 (USD)
港幣 (HKD)
英鎊 (GBP)
澳幣 (AUD)
加拿大幣 (CAD)
新加坡幣 (SGD)
瑞士法郎 (CHF)
日圓 (JPY)
南非幣 (ZAR)
瑞典幣 (SEK)
紐元 (NZD)
泰幣 (THB)
菲國比索 (PHP)
印尼幣 (IDR)
歐元 (EUR)
韓元 (KRW)
越南盾 (VND)
馬來幣 (MYR)
人民幣 (CNY)
